In [259]:
import requests
from weakref import ref as _wref
from collections import OrderedDict as OD
from copy import deepcopy
import shutil

from ruamel.yaml import YAML
yaml = YAML(typ='safe')   # default, if not specfied, is 'rt' (round-trip)

def yload(fp):
    if isinstance(fp, dict):
        return fp  # yaml already parsed
    if isinstance(fp, str):
        fp = open(fp, 'r')
    try:
        return yaml.load(fp)
    finally:
        fp.close()

In [316]:
class HasCTXBase():
    def __init__(self, ctx):
        self._ctx = _wref(ctx)
    @property 
    def ctx(self):
        return self._ctx()

class ASchema(HasCTXBase):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx)
        self.name = name
        self.description = schema.get('description', "")
        self.type = schema.get('type', None)
        self.default = schema.get('default', None)
        
    def get_fields(self):
        raise NotImplementedError
        

class ABasicSchema(ASchema):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx, name, schema)
        self.schema = schema
        
    def resolve(self, path=""):
        pass
    
    def get_fields(self):
        return [(self.name, self.type)]
        
class AObjectSchema(ASchema):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx, name, schema)
        self.properties = deepcopy(schema['properties'])
        self.type = 'object'
        
    def resolve(self, path=""):
        props = self.properties.copy()
        for name, item in props.items():
            if is_unresolved_ref(item):
                path = item['$ref']
                self.properties[name] = self.ctx.schemas[path]
            else:
                self.properties[name] = self.ctx.parse_a_schema(name, item)
        
class AArraySchema(ASchema):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx, name, schema)
        self.items = schema['items']
        self.type = 'array'
        
    def get_contents(self):
        return self.items
    
    def resolve(self, path=""):
        if is_unresolved_ref(self.items):
            path = self.items['$ref']
            self.items = self.ctx.schemas[path]
            
def _properties_to_schema(ctx, props):
    ret = OD()
    for pk, pv in props.items():
        sch = ctx.parse_a_schema(pk, pv)
        ret[pk] = sch
    return ret

def _parse_object_schema(ctx, oschema):
    contents = OD()
    required = []
    props = {}
    for item in oschema:
        for key, value in item.items():
            if key == "$ref":
                # inheritance
                if value in ctx.schemas:
                    sch = ctx.schemas[value]
                    name = value.split("/")[-1]
                else:
                    parts = value.split("/")[1:]
                    obj = ctx.yaml
                    for p in parts:
                        obj = obj[p]
                    sch = ctx.parse_a_schema(p, obj, value)
                    name = p
                sch._is_inherit = True
                contents[name] = sch
            elif key == 'properties':
                contents.update(_properties_to_schema(ctx, value))
            elif key == 'required':
                for p in value:
                    required.append(p)
            elif key in {'description', 'example', 'nullable', 'readOnly'}:
                props[key] = value
            else:
                raise ValueError("%s %s"%(key, oschema))
    return contents, required
        
class AComboSchema(ASchema):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx, name, schema)
        
        for k, v in schema.items():
            if k == 'allOf' or k == 'anyOf' or k == 'oneOf':
                type = k
                values = v
            else:
                setattr(self, k, v)
        self.type = type
        self.options, self.required = _parse_object_schema(ctx, schema[self.type])
        
        
    def resolve(self, bpath="#/components/schemas/"):
        ops = self.options.copy()
        for name, item in ops.items():
            if is_unresolved_ref(item):
                path = bpath + name
                sch = self.ctx.schemas[path]
                self.options[name] = sch
            else:
                sch = item
            
            if hasattr(sch, "_is_inherit"):
                sch.resolve(bpath)
                del self.options[name]
                if sch.get_type() == AObjectSchema:
                    it = sch.properties.items()
                elif sch.get_type() == AComboSchema:
                    it = sch.options.items()
                elif sch.get_type() == ABasicSchema:
                    self.options[name] = sch
                    continue
                elif sch.get_type() == AArraySchema:
                    self.options["items"] = sch.items
                    continue
                else:
                    raise ValueError(sch.name)
                for k, v in it:
                    self.options[k] = v

class AUnresolvedRefSchema(ASchema):
    def __init__(self, ctx, name, schema):
        super().__init__(ctx, name, schema)
        self.ref = schema['$ref']
        assert len(schema) == 1
        
    def get_resolved(self):
        return self.ctx.schemas[self.ref]
    
    def resolve(self, path=""):
        raise ValueError("We shouldn't be here!!!")
    
                
class ASchemaContainer():
    def __init__(self, ctx, name, schema, klass):
        self._obj = klass(ctx, name, schema)
        self._resolved = False
        
    def __getattr__(self, attr):
        return getattr(self._obj, attr)
    
    def resolve(self, path="#/components/schemas/"):
        if self._resolved:
            return
        if isinstance(self._obj, AUnresolvedRefSchema):
            self._obj = self._obj.get_resolved()._obj
        else:
            self._obj.resolve(path)
        self._resolved = True
        
    def get_type(self):
        return type(self._obj)
        
def is_unresolved_ref(obj):
    if isinstance(obj, ASchemaContainer):
        return False
    rv = "$ref" in obj
    if rv: 
        assert len(obj) == 1
    return rv

class APathBase(HasCTXBase):
    def __init__(self, ctx, path, props):
        super().__init__(ctx)
        self.path = path

class APath(APathBase):
    def __init__(self, ctx, path, props):
        super().__init__(ctx, path, props)
        self.attrib = OD()
        self.properties = OD()
        self.methods = OD()
        self.extract_props(props)
        
    def extract_props(self, props):
        
        for key, val in props.items():
            if key.startswith("x-"):
                self.attrib[key] = val
            elif key.upper() in ('GET', 'POST', 'PUT', 'DELETE'):
                key = key.upper()
                method = AMethod(self.ctx, self, key, val)
                self.methods[key] = method
            elif key == 'parameters':
                self.attrib['parameters'] = val
            else:
                raise ValueError(key)
                
    def resolve(self):
        for m in self.methods.values():
            m.resolve()
            
                
class AMethodBase(HasCTXBase):
    def __init__(self, ctx, pathobj, method, props):
        super().__init__(ctx)
        
class AMethod(AMethodBase):
    def __init__(self, ctx, pathobj, method, props):
        super().__init__(ctx, pathobj, method, props)
        self.pathobj = pathobj
        self.method = method
        props = props.copy()  # shallow copy
        self.summary = props.pop('summary', '')
        self.description = props.pop('description', '')
        self.func_name = props.pop('operationId', '')
        self.security = self._get_security(props)
        self.description = props.pop('description', '')
        self.params = props.pop('parameters', [])
        self.responses = props.pop('responses')
        self.tags = props.pop('tags')
        
        self.exc = props.pop('x-exegesis-controller', None)
        self.perforce_bodyname = props.pop('x-perforce-bodyname', None)
        
        self.requestbody = props.pop("requestBody", None)
        
        if props:
            raise ValueError("%s: %s \n%s"%(method, pathobj.path, props))
        
    def _get_security(self, props):
        sec = []
        for kv in props.pop('security', {}):
            for key,value in kv.items():
                assert not value
                sec.append(key)
        return sec
    
    def _parse_params(self, params):
        rv = []
        for kv in params:
            for k,v in kv.items():
                if k == '$ref':
                    param = self.ctx.parameters[v]
                else:
                    assert False, params
                    param = ctx.parse_a_schema(ctx, k, v)
                rv.append(param)
        return rv
    
    def resolve(self):
        inherited = self.pathobj.attrib.get('parameters', [])
        ihparams = self._parse_params(inherited)
        myparams = self._parse_params(self.params)
        self.params = ihparams + myparams
        
        # todo: request body for post
        

class AParameter(HasCTXBase):
    def __init__(self, ctx, name, props):
        super().__init__(ctx)
        props = props.copy()
        self.name = props.pop('name')
        self.loc = props.pop('in')
        self.description = props.pop('description')
        self.required = props.pop('required', False)
        self.explode = props.pop('explode', None)
        
        schema = props.pop('schema')
        self.schema = ctx.parse_a_schema(name, schema)
        
        props.pop('example', "")
        assert not props, (name, props)
        
    def resolve(self):
        self.schema.resolve()
        
class ARequestBody(HasCTXBase):
    def __init__(self, ctx, name, props):
        super().__init__(ctx)
        props = props.copy()
        content = props.pop('content')
        description = props.pop('description')
        required = props.pop('required')
        assert not props, props

In [317]:
def tag2class(tag):
    return tag.title().replace(" ","")

class ClassProxy():
    def __init__(self, name):
        self.name = name
        self.methods = []

class Context():
    def __init__(self, y):
        self.schemas = OD()
        self.paths = OD()
        self.parameters = OD()
        self.classes = OD()
        self.yaml = y
        self._all_schemas = []
        
    def classify(self):
        for p in self.paths.values():
            for m in p.methods.values():
                for t in m.tags:
                    clsn = tag2class(t)
                    if clsn not in self.classes:
                        self.classes[clsn] = ClassProxy(clsn)
                    self.classes[clsn].methods.append(m)                
        
    def parse_parameters(self):
        for name, props in self.yaml['components']['parameters'].items():
            path = "#/components/parameters/" + name
            if path not in self.parameters:
                self.parameters[path] = AParameter(self, name, props)
        for p in self.parameters.values():
            p.resolve()
        
    def parse_schemas(self):
        for name, schema in self.yaml['components']['schemas'].items():
            path = "#/components/schemas/" + name
            if path not in self.schemas:
                sch = self.parse_a_schema(name, schema)
                self.schemas[path] = sch
        for sch in self._all_schemas:
            sch.resolve()
        
    def parse_a_schema(self, name, schema, path=""):
        if path and path in self.schemas:
            return self.schemas[path]
        if 'properties' in schema:
            sch = AObjectSchema
        elif 'items' in schema:
            sch = AArraySchema
        elif any(i in schema for i in ('allOf', 'anyOf', 'oneOf')):
            sch = AComboSchema
        elif '$ref' in schema:
            sch = AUnresolvedRefSchema
        elif 'type' in schema:
            sch = ABasicSchema
        else:
            raise ValueError("%s: %s"%(name, schema))

        rv = ASchemaContainer(self, name, schema, sch)
        self._all_schemas.append(rv)
        return rv
    
    def parse_paths(self):
        for path, props in self.yaml['paths'].items():
            if path not in self.paths:
                self.paths[path] = APath(self, path, props)
        for path in self.paths.values():
            path.resolve()
            
    def sanity_check_paths(self):
        for p in self.paths.values():
            for m in p.methods.values():
                for par in m.params:
                    if par.loc == "path" and "{%s}"%par.name not in p.path:
                        print("Warning! Can't find param '{%s}' in path '%s'" %( par.name, p.path))
                        
    def fix_params(self):
        for p in self.paths.values():
            for m in p.methods.values():
                m.path_params = []
                m.query_params = []
                m.body_param = None
                for par in m.params:
                    if par.loc == "query":
                        m.query_params.append(par)
                    elif par.loc == "path":
                        m.path_params.append(par)
                    else:
                        assert par.loc == "body"
                        assert par.body_param == None
                        m.body_param = par
                m.path_params.sort(key=lambda par: p.path.index("{%s}"%par.name))
                    
        
def parse_yaml(y):
    global ctx
    ctx = Context(y)
    ctx.parse_schemas()
    ctx.parse_parameters()
    ctx.parse_paths()
    ctx.classify()
    ctx.sanity_check_paths()
    ctx.fix_params()
    return ctx

In [318]:
try:
    y
except NameError:
    thefile = "c:/users/nathan/downloads/helixalmrestapi.yaml"
    y = yload(thefile)
ctx = parse_yaml(y)

In [319]:
import jinja2

def method_param_string(method):
    return paramstring_GET(method)

def paramstring_GET(method):
    s = ", ".join(p.name for p in method.params if p.loc in ("path", "query"))
    if s:
        s = "self, %s"%s
    else:
        s = "self"
    return s

def body_arg(method):
    if method.method in ("POST", "PUT"):
        return "body"
    return "None"

def param_string2(method):
    params = []
    
    params.append("self")
    if method.method in ("POST", "PUT"):
        params.append("body")
        
    for p in method.params:
        if p.loc not in ("path", "query"):
            continue
        if p.schema.default is not None:
            s = "%s=_NoArg" % p.name
        elif p.loc == "path":
            s = p.name
        else:
            s = '%s=_NoArg'%p.name
        params.append(s)
    
    return ", ".join(params)
        
def query_params(method):
    return [p for p in method.params if p.loc == 'query']
    
def queryif(p, dname):
    return "if %s != _NoArg: %s[\"%s\"] = %s"%(p.name, dname, p.name, p.name)

def param_string_format(method):
    s=", ".join ("%s=%s"% (p.name, p.name) for p in method.params if p.loc == "path")
    if s:
        s = ".format(%s)"%s
    return s

def format_schema_value(pv):
    if pv.get_type() == ABasicSchema:
        return pv.default
    elif pv.get_type() in (AObjectSchema, AComboSchema):
        return pv.name + "()"
    else:
        assert pv.get_type() == AArraySchema, (pv.get_type(), pv.name)
        return "[]"

basepath = "C:\\Users\\Nathan\\Documents\\Personal\\test\\openapiclient\\"
template_path = basepath + "templates\\"
out_path = basepath + "out\\"
os.makedirs(out_path, exist_ok=True)
    
templateLoader = jinja2.FileSystemLoader(searchpath=template_path)
templateEnv = jinja2.Environment(loader=templateLoader)

templateEnv.globals.update({
    'param_string2': param_string2,
    'param_string_format': param_string_format,
    "query_params": query_params,
    "queryif": queryif,
    "format_schema_value": format_schema_value,
})

def render_template(file, props):
    template = templateEnv.get_template(file)
    return template.render(**props)

def load_template(file, props):
    try:
        return render_template(file, props)
    except jinja2.TemplateSyntaxError as e:
        print("Error loading template '%s'"%file)
        print(e.message)
        print(e.lineno)
        raise Exception(e.message) from None
        
def get_output_filename(ppath, path, file):
    dir = os.path.join(ppath, path)
    dir = os.path.normpath(dir)
    os.makedirs(dir, exist_ok=True)
    if not file.endswith(".py"):
        file += ".py"
    return os.path.join(dir, file)

def handle_file(props, ppath):
    file = props['template']
    path = props.get('path', ".")
    out = get_output_filename(ppath, path, props['filename'])
    result = load_template(file, props['env'])
    with open(out, 'w') as f:
        f.write(result)
        
def handle_project(props):
    path = props['base_path']
    project = props['project_name']
    ppath = os.path.join(path, project)
    ppath = os.path.normpath(ppath)
    for file in props['file_list']:
        handle_file(file, ppath)

In [326]:
# ctx = parse_yaml(y)


api_client = {
    "path": ".",
    "template": "ApiClient.py.jinja2",
    "filename": "ApiClient",
    "env": {
        "base_url": "https://pbsbiotech.helixalm.cloud:8443",
        "server_path": ctx.yaml['servers'][0]['url'],
        "security_class": "Security",
        "ctx": ctx
   }
}

files = {
    "base_path": "C:\\users\\nathan\\documents\\personal\\test",
    "project_name": "helixalmclient",
    "file_list": [
        api_client,
        {
            "path": ".",
            "filename": "Security",
            "template": "BasicAccessTokenSecurityTemplate.py.jinja2",
            "env": {
                "auth_class": ctx.classes["Security"]
            }
        }, 
        {
            "path": ".\paths",
            "filename": "BasePath.py",
            "template": "BasePath.py.jinja2",
            "env": {}
        },
    ]
}

# paths
for cls in ctx.classes.values():
    if cls.name == "Security": 
        continue
    else:
        files['file_list'].append({
            'path': ".\paths",
            'filename': cls.name + ".py",
            'template': 'BasicPathTemplate.py.jinja2',
            'env': {
                'ctx': ctx,
                'cls': cls
            }
        })
        
def _import_schema(pk):
    return "from models.%s import %s" % (pk.name, pk.name)
        
# schemas
for s in ctx.schemas.values():
    t = s.get_type()
    
    if t == ABasicSchema:
        continue
    elif t == AObjectSchema:
        template = "ObjectSchema.py.jinja2"
        imports = []
        for pk, pv in s.properties.items():
            if pv.get_type() in (AObjectSchema, AComboSchema):
                imports.append(_import_schema(pv))
        
    elif t == AComboSchema:
        template = "ComboSchema.py.jinja2"
        imports = []
        for pk, pv in s.options.items():
            if pv.get_type() in (AObjectSchema, AComboSchema):
                imports.append(_import_schema(pv))
    elif t == AArraySchema:
        continue
    else:
        raise ValueError(t)
        
    files['file_list'].append({
        "path": ".\models",
        "filename": s.name + ".py",
        "template": template,
        "env": {
            "ctx": ctx,
            "schema": s,
            "imports": imports
        }
    })

handle_project(files)

In [227]:
import subprocess

proc = subprocess.Popen(["python", outf], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
out = proc.communicate()[0]
out = out.decode()
print(out)

python: can't open file 'C:\Users\Nathan\Documents\Personal\test\openapiclient\out\helixclient.py': [Errno 2] No such file or directory



In [251]:
ctx.schemas["#/components/schemas/snapshots"].get_type()

KeyError: '#/components/schemas/snapshots'